In [1]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
import numpy as np
def json_prettify(json_res):
    return json.dumps(json_res.json(), indent=2)

def get_auth():
    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'

    headers = {
        'Content-Type': 'application/json-rpc'
    }
    auth_query = {
        "jsonrpc": "2.0",
        "method": "user.login",
        "params": {
            "user": "QAC",
            "password": "QAC12345"
        },
        "id": 1
    }


    response = requests.post(url, headers=headers, data=json.dumps(auth_query)).json()
    return response['result']
auth = get_auth()

# def to_timestamp(unix_timestamp):
#     # Convert Unix timestamp to a datetime object
#     timestamp = datetime.utcfromtimestamp(int(unix_timestamp))
#     timestamp = timestamp + timedelta(hours=7)
#     # Format the datetime object as a human-readable timestamp
#     formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:00')

#     return formatted_timestamp

def to_timestamp(unix_timestamp):
    # Convert Unix timestamp to a datetime object
    timestamp = datetime.utcfromtimestamp(int(unix_timestamp))
    timestamp = timestamp + timedelta(hours=7)
    # Format the datetime object as a human-readable timestamp
    formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:00')

    return formatted_timestamp

In [2]:
# a = pd.read_json('host-list.json')
# a.to_excel('host-list.xlsx', index=False)

# b = pd.read_json('item-res.json')
# b.to_excel('item-res.xlsx', index=False)

In [3]:
def get_data(auth, history, time_start, time_till, item_id):
    time_start = int(datetime.strptime(time_start, "%Y-%m-%d %H:%M:%S").timestamp())
    time_till = int(datetime.strptime(time_till, "%Y-%m-%d %H:%M:%S").timestamp())

    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'
    headers = {
        'Content-Type': 'application/json-rpc',
    }

    data_query = {
            "jsonrpc": "2.0",
            "method": "history.get",
            "params": {
                "output": "extend",
                "history": history,
                "itemids": item_id,
                "sortfield": "clock",
                "sortorder": "DESC",
                "time_from": time_start,
                "time_till": time_till,
                "limit": 50000
            },
            "auth": auth,
            "id": 1
        }

    response = requests.post(url, headers=headers, data=json.dumps(data_query)).json()
    
    df = pd.DataFrame(response['result'])

    return df

In [4]:
def date_to_unix(start_date, end_date):
    # Convert datetime objects to Unix timestamps
    start_date = int(start_date.timestamp())
    end_date = int(end_date.timestamp())

    # Add 7 hours to the Unix timestamps
    start_unix = start_date + 7 * 3600
    end_unix = end_date + 7 * 3600

    return start_unix, end_unix

def fill_null(dataframe, col):
    for i in range(len(dataframe)):
        for column in dataframe[col]:
            if pd.isnull(dataframe.at[i, column]):
                if i == 0:
                    num = 0
                    while num < len(dataframe)-1 and pd.isnull(dataframe.at[num, column]):                    
                        num +=1
                    next_value = dataframe.at[num, column]
                    dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = next_value
                    dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
                else:
                    prev_value = dataframe.at[i - 1, column]
                    dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = prev_value
                    dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
    return dataframe

In [5]:
# PAC
df_pac = pd.read_excel('rekap DCI (PAC).xlsx')

In [7]:
# PAC
time_start = "2024-04-01 00:00:00"
time_till = "2024-04-30 23:59:59"

base_dir = './result/Tambahan/DCI tambahan Apr 2024/'

timestamp_list = pd.date_range(start=time_start, end=time_till, freq='1T')
summary = pd.DataFrame(columns=['hostname','temp_id', 'total_temp','hum_id','total_hum'])

for index, data in df_pac.iterrows():    
    hostname = data['hostname']
    filename= data['filename']
    item_id = data['unit_operation_state_id']
    history = data['unit_operation_state_hist']

    raw_state = pd.DataFrame(columns=['itemid','clock','value','ns'])
    raw_state_data = get_data(auth, history, time_start, time_till, item_id)


    raw_state = pd.concat([raw_state, raw_state_data], axis=0)
    raw_state['date'] = raw_state['clock'].apply(to_timestamp)
    raw_state = raw_state.groupby(by='date').max().reset_index()


    result = pd.DataFrame(columns=['hostname','date'])
    result['date'] = timestamp_list
    result['date'] = result['date'].astype('string')
    result = pd.merge(result, raw_state[['date','value']], on=['date'], how='left')

    result['hostname'] = hostname
    result[['value_null']] = ''
    
    final_data = fill_null(result, ['value'])
    final_data['value'] = final_data['value'].apply(lambda x: 1 if "on" in x else 0)
    final_data.to_csv(str(base_dir+filename+'.csv'), index=False)
    print(hostname)

GTI_GD31_PAC3.1
GTI_GD31_PAC3.11
GTI_GD31_PAC3.2
GTI_GD31_PAC3.3
GTI_GD31_PAC3.4
GTI_GD31_PAC3.5
GTI_GD31_PAC3.6
GTI_GD31_PAC3.7
GTI_GD31_PAC3.8
GTI_GD31_PAC3.9
GTI_GD31_PAC3_10
GTI_GD41_PAC4.1
GTI_GD41_PAC4.2
GTI_GD41_PAC4.3
GTI_GD41_PAC4.4
GTI_GD41_PAC4.5
GTI_GD41_PAC4.6
GTI_GD41_PAC4.7
GTI_GD41_PAC4.8


In [74]:
df_pac[9:10]

,hostname,filename,host_id,unit_operation_state_id,unit_operation_state_hist
9,GTI_GD31_PAC3.9,PAC_3_9,11084,40494,4
